# Using CAVE with AutoPyTorch

AutoPyTorch aims at building a framework for automated neural-network-configuration. Currently it supports [BOHB](https://github.com/automl/HpBandSter) for hyperparameter search.
CAVE integrates AutoPyTorch, building on it's function for further insights and visualizations.
This notebook provides an exemplary pipeline for using CAVE on / with AutoPyTorch.

We will generate some AutoPyTorch-Output. You can use your own AutoPyTorch-routine here, we will use the openml-tasks, inspired by [AutoPyTorch's tutorial notebook](https://github.com/automl/Auto-PyTorch/blob/master/examples/basics/Auto-PyTorch%20Tutorial.ipynb).

In [1]:
log_dir = "logs/apt-cave-notebook/"

In [2]:
from autoPyTorch import AutoNetClassification
import pandas as pd
import numpy as np
import os as os
import openml
import json
from ConfigSpace.read_and_write import json as pcs_json


task = openml.tasks.get_task(task_id=31)

X, y = task.get_X_and_y()
ind_train, ind_test = task.get_train_test_split_indices()
X_train, Y_train = X[ind_train], y[ind_train]
X_test, Y_test = X[ind_test], y[ind_test]

autonet = AutoNetClassification(config_preset="full_cs", result_logger_dir=log_dir)

# Get data from the openml task "Supervised Classification on credit-g (https://www.openml.org/t/31)"
task = openml.tasks.get_task(task_id=31)
X, y = task.get_X_and_y()
ind_train, ind_test = task.get_train_test_split_indices()
X_train, Y_train = X[ind_train], y[ind_train]
X_test, Y_test = X[ind_test], y[ind_test]

In [3]:
# Fit to find an incumbent configuration with BOHB
results_fit = autonet.fit(X_train=X_train,
                          Y_train=Y_train,
                          validation_split=0.3,
                          max_runtime=110,
                          min_budget=20,
                          max_budget=100,
                          refit=True)

In [4]:
# Save fit results as json
with open(os.path.join(log_dir, "results_fit.json"), "w") as f:
    json.dump(results_fit, f, indent=2)
    
# Also necessary information (can be migrated either to CAVE or (preferably) to autopytorch)
with open(os.path.join(log_dir, 'configspace.json'), 'w') as f:
    f.write(pcs_json.write(autonet.get_hyperparameter_search_space(X_train=X_train,
                                                                   Y_train=Y_train)))
with open(os.path.join(log_dir, 'autonet_config.json'), 'w') as f:
    json.dump(autonet.get_current_autonet_config(), f, indent=2)

We can then spin up CAVE and hand it the output, as well as the autonet-instance. That way, CAVE can refit the incumbents and we can investigate the evolution of the network a bit closer.

In [5]:
from cave.cavefacade import CAVE

# The information in the autonet-bundle needs to be logged and loaded eventually (or all necessary logging reliably triggered in apt itself)
autonet_bundle = {'autonet': autonet,
                  'X_train': X_train,
                  'Y_train': Y_train,
                 }

cave = CAVE([log_dir],      # List of folders holding results
            "cave_output",  # Output directory
            ['.'],          # Target Algorithm Directory (only relevant for SMAC)
            file_format="APT",
            autonet=autonet_bundle)

19:56:53 Getting attr __spec__ of LazyModule instance of emcee
19:56:53 Getting attr Kernel of LazyModule instance of skopt.learning.gaussian_process.kernels
19:56:53 Getting attr __name__ of LazyModule instance of skopt.learning.gaussian_process.kernels
19:56:53 Getting attr GaussianProcessRegressor of LazyModule instance of skopt.learning.gaussian_process
19:56:53 Getting attr __name__ of LazyModule instance of skopt.learning.gaussian_process
19:56:53 Getting attr Kernel of LazyModule instance of skopt.learning.gaussian_process.kernels
19:56:53 Getting attr __name__ of LazyModule instance of skopt.learning.gaussian_process.kernels
19:56:53 Getting attr GaussianProcessRegressor of LazyModule instance of skopt.learning.gaussian_process
19:56:53 Getting attr __name__ of LazyModule instance of skopt.learning.gaussian_process
19:56:53 Loaded backend agg version unknown.


Q: should CAVE even get an autonet-instance? is all relevant information saved with info about the autonet-instance? would be nicer if there simply was some sort of scenario-file (which is partly/mostly covered by the results-dump)

In [6]:
cave.apt_overview()

result_logger_dir,logs/apt-cave-notebook/
validation_split,0.3
max_runtime,110
min_budget,20
max_budget,100
log_level,warning
hyperparameter_search_space_updates,None
categorical_features,None
dataset_name,None
run_id,0
task_id,-1


Other analyzers also run on the APT-data:

In [7]:
cave.overview_table()

In [8]:
cave.compare_default_incumbent()

,Default,Incumbent
--------------- Changed parameters: ---------------,-----,-----
CreateDataLoader:batch_size,126,416
InitializationSelector:initialization_method,sparse,default
InitializationSelector:initializer:initialize_bias,Yes,Zero
LearningrateSchedulerSelector:lr_scheduler,cosine_annealing,adapt
NormalizationStrategySelector:normalization_strategy,none,maxabs
PreprocessorSelector:preprocessor,fast_ica,kernel_pca
ResamplingStrategySelector:over_sampling_method,none,random
ResamplingStrategySelector:target_size_strategy,median,upsample
ResamplingStrategySelector:under_sampling_method,none,random
